In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/raw/data.csv')

In [3]:
df.head()

,model_name,model_year,kms_driven,owner,location,price
0,Hero CBZ X-Treme Rear Disc,2014 model,"26,916 kms",1st Owner,Lucknow,"40,000"
1,Bajaj Pulsar 220 DTS-i Standard,2017 model,"8,500 kms",1st Owner,Mumbai,"90,000"
2,Bajaj Discover 135 Standard,2011 model,"40,000 kms",1st Owner,Kolkata,"25,000"
3,Hero Honda Splendor Plus Drum,2011 model,"40,000 kms",1st Owner,Bangalore,"30,000"
4,Bajaj Platina 100 ES Drum,2018 model,"18,000 kms",1st Owner,Baran,"41,000"


In [4]:
df.tail(10)

,model_name,model_year,kms_driven,owner,location,price
6562,Royal Enfield Bullet Electra 350cc,2015,"40,822 Km",First Owner,Areraj,"\n 90,001\n"
6563,Suzuki Intruder 150cc,2018,"19,000 Km",First Owner,Jajpur,"\n 68,200\n"
6564,Bajaj Pulsar AS150,2015,"12,566 Km",First Owner,Dhanbad,"\n 54,999\n"
6565,Suzuki Gixxer 150cc,2018,"10,815 Km",First Owner,Pune,"\n 75,000\n"
6566,KTM RC 200cc,2018,"11,000 Km",First Owner,Indore,"\n 1,60,000\n"
6567,Hero Impulse 150cc,2016,"40,000 Km",First Owner,Mumbai,"\n 35,000\n"
6568,Royal Enfield Classic Chrome 500cc,2013,"10,000 Km",First Owner,Anand,"\n 1,10,000\n"
6569,Bajaj Avenger Street 220,2017,"16,462 Km",Second Owner,Pune,"\n 60,000\n"
6570,Royal Enfield Classic Desert Storm 500cc,2013,"13,000 Km",Second Owner,Jaipur,"\n 1,04,000\n"
6571,Honda CBR 250R,2016,"17,000 Km",Second Owner,Chennai,"\n 95,999\n"


In [5]:
df.shape

(6572, 6)

## Data Cleaning

In [6]:
# remove whitespace from data
for col in df.columns:
    df[col] = df[col].apply(lambda x: x.strip() if x else x)

In [7]:
# Remove commas from kms_driven and price columns

def clean_kms_driven(val):
    if not val:
        return ''

    val = val.lower()

    if 'kms' in val:
        val = val.replace('kms','')

    if 'km' in val:
        val = val.replace('km','')

    return val.replace(',','')


def clean_price(val):
    """
    1. remove currency symbol.
    2. remove commas
    3. fix the val which is represented as lakh
    """
    if not val:
        return ''

    val = val.replace('₹','')

    val = val.replace(',','')

    if 'lakh' in val.lower():
        val = val.lower().replace('lakh','')
        val = float(val)
        return val * 100000

    return val
    

df['kms_driven'] = df['kms_driven'].apply(clean_kms_driven)

df['price'] = df['price'].apply(clean_price)

In [8]:
# Let's make the model_year columm as number only by removing the 'model' string
df['model_year'] = df['model_year'].str.replace('model','')

In [9]:
# location feature have comma separated values
# need to make it a single value, for that let's take
# the last value as it is the main city name

def clean_location(val):
    if not val:
        return ''
    
    return val.split(',')[-1].strip()

df['location'] = df['location'].apply(clean_location)

In [10]:
df.location.value_counts()[:21]

Delhi         1762
Bangalore      521
Mumbai         493
Hyderabad      301
Jaipur         240
New Delhi      226
Pune           178
 Mumbai        165
Chennai        158
Ahmedabad      149
 New Delhi     147
Kolkata        132
 Bangalore     126
Chandigarh     105
Gurgaon        100
 Kolkata        75
 Hyderabad      66
Noida           51
Thane           51
 Ahmedabad      50
Ghaziabad       45
Name: location, dtype: int64

In [11]:
# Fix owner column values
# it is represented in two different way one as 1st owner and other in words like first owner
# let's make them homogeneous

def clean_owner(val):
    if not val:
        return val

    owners = {
        '1st Owner': '1st',
        '2nd Owner': '2nd',
        '3rd Owner': '3rd',
        '4th Owner': '4th',
        '5th Owner': '5th',
        'First Owner': '1st',
        'Second Owner': '2nd',
        'Third Owner': '3rd',
        'Fourth Owner': '4th',
        'Fourth Owner Or More': '5th',
    }

    return owners.get(val,'')

df['owner'] = df['owner'].apply(clean_owner)

In [12]:
df.sample(10)

,model_name,model_year,kms_driven,owner,location,price
4532,TVS Apache RTR 160 4V FI,2019,95000,2nd,Mumbai,61000\n 60390
4740,KTM Duke 200cc,2015,58000,1st,Panruti,95000
1596,Bajaj Avenger Cruise 220 STD BS VI,2017,20000,1st,New Delhi,70000
5753,Hero Karizma R 223cc,2009,96000,1st,Mumbai,25000
6000,TVS Apache 150cc,2015,52000,1st,Hyderabad,49000
4474,Royal Enfield Classic 350cc,2016,11000,2nd,Delhi,140000\n 138600
4933,Bajaj Pulsar 220cc,2014,31773,1st,Bangalore,48000
1757,Royal Enfield Bullet 350 STD,2015,11000,1st,Rohtak,90000
4398,Yamaha FZ25 250cc,2018,15000,1st,Pune,110000
5090,Royal Enfield Bullet 500cc,2016,24000,1st,Delhi,150000


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6572 entries, 0 to 6571
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   model_name  6572 non-null   object
 1   model_year  6572 non-null   object
 2   kms_driven  6572 non-null   object
 3   owner       6572 non-null   object
 4   location    6572 non-null   object
 5   price       6572 non-null   object
dtypes: object(6)
memory usage: 308.2+ KB


In [14]:
df['model_year'] = pd.to_numeric(df['model_year'],errors='coerce')
df['kms_driven'] = pd.to_numeric(df['kms_driven'],errors='coerce')
df['price'] = pd.to_numeric(df['price'],errors='coerce')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6572 entries, 0 to 6571
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   model_name  6572 non-null   object 
 1   model_year  6572 non-null   int64  
 2   kms_driven  6569 non-null   float64
 3   owner       6572 non-null   object 
 4   location    6572 non-null   object 
 5   price       6319 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 308.2+ KB


In [16]:
df.duplicated().sum()

845

In [17]:
df = df[~df.duplicated()]

In [18]:
df.shape

(5727, 6)

## Handle Missing Values

In [19]:
df.isnull().sum()

model_name      0
model_year      0
kms_driven      3
owner           0
location        0
price         207
dtype: int64

In [20]:
df[df.isnull()]

,model_name,model_year,kms_driven,owner,location,price
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
6567,NaN,NaN,NaN,NaN,NaN,NaN
6568,NaN,NaN,NaN,NaN,NaN,NaN
6569,NaN,NaN,NaN,NaN,NaN,NaN
6570,NaN,NaN,NaN,NaN,NaN,NaN


### It looks like there are empty rows present,let's drop them.

In [21]:
df.dropna(inplace=True)

In [22]:
df.shape

(5517, 6)

In [23]:
df.to_csv('../data/processed/data.csv',index=False)